In [1]:
from split_data import split_and_save

split_and_save()

validation data saved
test data saved
success! c:


In [2]:
# %load cleaning_data.py
import pandas as pd
import re
from langdetect import detect
from collections import Counter
import numpy as np
import string
from unicodedata import category
import sys
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#Languages_detection
def det_lang(df, text):
    def detect_language(text):
        try:
            lan = detect(text)
        except:
            lan = 'unknown'
        return lan

    df['language'] = df['text'].apply(detect_language)
    return df

        
    

#delete stopwords
def delete_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
    
   
#removing noise
def clean_text(text, punctuation_chars):
    text = text.lower()

    # remove punctuation
    text = text.translate(str.maketrans('', '', ''.join(punctuation_chars)))
    # remove digits
    text = text.translate(str.maketrans('', '', string.digits))
    # remove all single characters
    pattern = r'(^| ).( |$)'
    text = re.sub(pattern, ' ', text)
    # remove multiple spaces
    text = re.sub(' +', ' ', text)
    return text


#text cleaning
def clean_df(df):
    df = df.fillna('')
    punctuation_chars = [chr(i) for i in range(sys.maxunicode)
                         if category(chr(i)).startswith("P")]
    df['text'] = df['text'].map(lambda x: clean_text(x, punctuation_chars))
    df['title'] = df['title'].map(lambda x: clean_text(x, punctuation_chars))
    #
    df['text']=df['text'].apply(delete_stopwords)
    df['title']=df['title'].apply(delete_stopwords)
    #Drops rows where the length of the 4th column is less than 30
    df = df[df['text'].str.len() >= 30]
    df=df.rename(columns={df.columns[0]: 'id'})
    df.dropna(subset=['title','text'], inplace=True)
    languag = det_lang(df,'text')

    
    
    return df

df = pd.read_csv('original_data.csv')
df = clean_df(df)
df.to_csv('modified_txt.csv', index=False)


    


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bfgam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#check the dataset
df.head(0)
df.shape

(67916, 5)

In [4]:
#let's count how many fake and true news has been written in each language#
counts = df.groupby(['language', 'Ground Label']).size()
print(counts)

language  Ground Label
af        fake                8
ar        fake               22
ca        fake               10
          true                2
cy        fake                3
da        fake               12
          true                3
de        fake              111
el        fake                4
en        fake            40330
          true            26869
es        fake              173
et        fake                2
fr        fake               58
          true                4
hr        fake               11
id        fake                4
it        fake               26
          true                1
nl        fake               13
no        fake               11
          true                1
pl        fake                5
pt        fake               12
ru        fake              203
sv        fake                3
sw        fake                4
tr        fake               10
zh-cn     fake                1
dtype: int64


In [5]:
#remove other languages to neutralize noise in the dataset
en_df=df[df['language']=='en']
en_df.shape


(67199, 5)

In [6]:
#drop the useless columns
en_df=en_df.drop(['id','language'],axis=1)
en_df.head()

,title,text,Ground Label
0,ann coulter make believes gay friends make rac...,hard believe donald trump sizable amount suppo...,fake
2,add dirty money laundering obama billion iran ...,member house intelligence committee accusing o...,fake
3,republicans obamacare repeal going get done,washington reuters house representatives repub...,true
4,trump possible daca deal says border wall woul...,washington reuters us president donald trump s...,true
5,trump administration force peace james pinkerton,originally appeared american conservative dona...,fake


In [9]:
#let's combine useful text data
en_df['full_text']=en_df['title'] + ' ' + en_df['text']
en_df=en_df.drop(['title','text'],axis=1)

en_df.head(1)

,Ground Label,full_text
0,fake,ann coulter make believes gay friends make rac...


In [10]:
#text tokenizing
import nltk
nltk.download('all')

en_df['full_text']=en_df['full_text'].apply(nltk.word_tokenize)
#tokenized joint texts
print(en_df.head(3))

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\bfgam\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\bfgam\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\bfgam\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\bfgam\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\bfgam\AppData\Roaming\nltk_data...
[nltk_data]    | 

  Ground Label                                          full_text
0         fake  [ann, coulter, make, believes, gay, friends, m...
2         fake  [add, dirty, money, laundering, obama, billion...
3         true  [republicans, obamacare, repeal, going, get, d...
